In [1]:
!pip install --upgrade pinecone pinecone-text pinecone-notebooks python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
# Load environment variables from a local .env file (if present)
load_dotenv()
# Read the API key from the environment. Create a .env file with PINECONE_API_KEY=your_key if needed.
api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    raise ValueError("PINECONE_API_KEY not set. Create a .env file with PINECONE_API_KEY=your_key or set the environment variable.")

In [3]:
!pip install langchain langchain-community
from langchain_community.retrievers import PineconeHybridSearchRetriever


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pinecone
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
# Initialize Pinecone client
pc = Pinecone(api_key=api_key)

# Create the Pinecone index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # chosen dimension of dense vector based on embedding technique (used later)
        metric="dotproduct", #sparse values supported only for dotproduct (for hybird search)
        spec = ServerlessSpec(cloud='aws', region='us-east-1')
    )


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
index  = pc.Index(index_name)
index

c:\Users\Avi\source\airepos\ai-rag-pinecone-demo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
!pip install sentence-transformers langchain-huggingface torch
# vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
test = embeddings.embed_query("hello world")
print(len(test), test[:5])

384 [-0.03447727859020233, 0.031023195013403893, 0.0067350007593631744, 0.02610897831618786, -0.03936203941702843]


In [8]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [9]:
sentences = [
    "In 2023 I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited London",
]

#tfidf values on these sentences
bm25_encoder.fit(sentences)

#store the values to a json file
bm25_encoder.dump("bm25_values.json")

#load the values from the json file
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 41.62it/s]


In [10]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,
                                          sparse_encoder=bm25_encoder,
                                          index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002063E65B8F0>, index=<pinecone.db_data.index.Index object at 0x000002063DA0E120>)

In [11]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


In [12]:
retriever.invoke("What about 2022?") #internally, it uses both dense and sparse search

[Document(metadata={'score': 0.471748412}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.234191418}, page_content='In 2023 I visited Paris'),
 Document(metadata={'score': 0.149339199}, page_content='In 2021, I visited London')]